In [1]:
import torch
import numpy as np

In [2]:
T=[[[float("NaN"),float("NaN")],[3.0,4.0]],[[2.0,2.0],[1.0,2.0]]]
V=torch.tensor(T)
print(V)
print(torch.norm(V,p=2,dim=2))

tensor([[[nan, nan],
         [3., 4.]],

        [[2., 2.],
         [1., 2.]]])
tensor([[   nan, 5.0000],
        [2.8284, 2.2361]])


In [19]:
V[V==V].reshape(3,2)

tensor([[3., 4.],
        [2., 2.],
        [1., 2.]])

In [21]:
T=[[[1.0,float("NaN")],[3.0,4.0]],[[2.0,2.0],[1.0,2.0]]]
V=torch.tensor(T)

In [28]:
Z=(V==V).all(axis=2)
print(Z)

V[Z]

tensor([[False,  True],
        [ True,  True]])


tensor([[3., 4.],
        [2., 2.],
        [1., 2.]])

In [35]:
T=[[[1.0,float("NaN")],[float("NaN"),float("NaN")]],[[2.0,2.0],[1.0,2.0]]]
V=torch.tensor(T)
XT=V

In [36]:
(XT!=XT).any(axis=2)*(XT==XT).any(axis=2)

tensor([[ True, False],
        [False, False]])

In [60]:
S=[[3.0,2.0],[1.0,2.0]]
g=torch.tensor(S).unsqueeze(-2).numpy()

Z=torch.ones(2).reshape((-1,1)).numpy()

print(np.transpose(np.kron(Z,g),axes=(0,1,2)))

[[[3. 2.]
  [3. 2.]]

 [[1. 2.]
  [1. 2.]]]


In [65]:
np.sum(g,axis=0)

array([[4., 4.]], dtype=float32)

In [66]:
D=torch.tensor(S)

V=torch.tensor([2.1,2.0])

print(torch.matmul(V,D))

tensor([8.3000, 8.2000])


In [5]:
def FiltreNanHomogène(XT,Y):
    """
    Prend en entrée XT (nb_patient,nb_visit_max,dim) et retourne X sous la forme (nb_visit,dim)

    Si un vecteur contient un Nan dans ses coordonnées on le retire

    """


    Select=((XT==XT).all(axis=2))*(Y==Y).all(axis=2)#fonctionne bien voir notebook test pour se convaincre
    
    return XT[Select],Y[Select]

In [9]:
X,Y=FiltreNanHomogène(V,V)

In [7]:
def Sub_sampling(X,k):
    """
    Prend X le tensor (nb_visite,dim) et sélectionne k points bien espacé renvoyé dans un tensor (k,dim)


    """
    Center,index=kmeans_plus_plus(X.numpy(), k)
    return torch.from_numpy(Center),index

In [27]:
X.repeat(3,1,1)

tensor([[[3., 4.],
         [2., 2.],
         [1., 2.]],

        [[3., 4.],
         [2., 2.],
         [1., 2.]],

        [[3., 4.],
         [2., 2.],
         [1., 2.]]])

In [34]:
def Matrix(X,Xgrand,meta_settings):
    """
    X est la donnée des points de controles un tensor de la forme (k,nb_dim) (k nombre de visite après subsampling), kernelname le nom du noyau à utiliser
    cette fonction renvoie la matrice K_X=(k(x_i,x_j)) i <nb_visit+1, j<k+1
    On a Xgrand (nb_visit,nb_dim) les points de controle sans subsampling
    """
    kernelname=meta_settings["kernelname"]
    sigma=meta_settings["sigma"]
    k=len(X)
    nb_visit=len(Xgrand)
    
    
    if kernelname=="RBF":#le calcul est fait sans approximations
        sigma=meta_settings["sigma"]
        

        PA1=Xgrand.repeat(k,1,1)
        PA2=X.repeat(nb_visit,1,1).permute(1,0,2)

        PA3=PA1-PA2

        K_X=torch.exp(-torch.norm(PA3,dim=2)**2/(2*sigma**2))
    else:
        raise ValueError("Le nom de noyau est mauvais ! ")

    return K_X

In [35]:
Matrix(X,A,{"sigma":2.0,"kernelname":"RBF"})

tensor([[1.0000, 0.5353],
        [0.5353, 1.0000],
        [0.3679, 0.8825]], dtype=torch.float64)

In [42]:
X1=X.repeat(2,1,1).permute(1,0,2)

X2=A.repeat(3,1,1)





print(A)

print(A.transpose(0,1))

tensor([[3., 4.],
        [2., 2.]], dtype=torch.float64)
tensor([[3., 2.],
        [4., 2.]], dtype=torch.float64)


In [20]:
def dist(data, centers):
    distance = np.sum((np.array(centers) - data[:, None, :])**2, axis = 2)
    return distance
def kmeans_plus_plus(X, k):
    '''Initialize one point at random.
    loop for k - 1 iterations:
        Next, calculate for each point the distance of the point from its nearest center. Sample a point with a 
        probability proportional to the square of the distance of the point from its nearest center.'''
    centers = []
    index=[]
    
    # Sample the first point
    initial_index = np.random.choice(range(X.shape[0]), )
    index.append(initial_index)
    centers.append(X[initial_index, :].tolist())
    
    print('max: ', np.max(np.sum((X - np.array(centers))**2)))
    
    # Loop and select the remaining points
    for i in range(k - 1):
        print(i)
        distance = dist(X, np.array(centers))
        
        if i == 0:
            pdf = distance/np.sum(distance)
            indexcour=np.random.choice(range(X.shape[0]), replace = False, p = pdf.flatten())
            centroid_new = X[indexcour]
            index.append(indexcour)
        else:
            # Calculate the distance of each point from its nearest centroid
            dist_min = np.min(distance, axis = 1)
            
            pdf = dist_min/np.sum(dist_min)
# Sample one point from the given distribution
            indexcour=np.random.choice(range(X.shape[0]), replace = False, p = pdf)
            centroid_new = X[indexcour]
            index.append(indexcour)
            

        centers.append(centroid_new.tolist())
        
    return np.array(centers),index

In [21]:
A,B=Sub_sampling(X,2)

print(A)

max:  13.0
0
tensor([[3., 4.],
        [2., 2.]], dtype=torch.float64)


In [74]:
def TransformationB(W,Control,meta_settings):
    """
    Prend en entrée la matrice des poids, et les points de contrôles "Control" ainsi que meta_settings pour avoir des
    informations sur le noyau, W de forme (nb_controle,nb_features). On renvoie la fonction associée pour update B.

    à finir et cheker
    """
    if meta_settings["kernelname"]=="RBF":
        sigma=meta_settings["sigma"]
        def function(x):
            nb_dm=len(x.shape)
            Control1=Control.unsqueeze(-1)
            KK=torch.exp(-torch.norm(Control-x,dim=2)**2/(2*sigma**2))
            
            Fin=torch.matmul(KK,W.double()).numpy()
           
            if len(Fin.shape)==0:
                return Fin.reshape(1,)
            else:
                return torch.from_numpy(np.sum(Fin,axis=1))
        return function
    else:
        raise ValueError("Le nom de noyau est mauvais ! ")

In [73]:
Control=A
meta_settings={"sigma":2.0,"kernelname":"RBF"}
W=torch.tensor([[2.1,3.0],[2.1,3.0]])

fonc=TransformationB(W,Control,meta_settings)

print(torch.matmul(W,W))
print(fonc(W))

tensor([[10.7100, 15.3000],
        [10.7100, 15.3000]])
tensor([0.7975, 0.8814], dtype=torch.float64)
torch.Size([2, 2])
[3.52571746 5.03673946]


AxisError: axis 1 is out of bounds for array of dimension 1